# **Single Cell Foundation models**



**The central dogma in biology**: information flows in one direction:
DNA → RNA → Protein

DNA is the library → RNA is the photocopy of a recipe → Protein is the dish made from it


**RNA sequencing (RNA-seq)** measures RNA levels in a sample  
**Single-cell RNA sequencing (scRNA-seq)** measures RNA in each individual cell


| Cell ID | Gene A | Gene B | Gene C |
|---------|--------|--------|--------|
| Cell 1  |   5    |   0    |   3    |
| Cell 2  |   1    |   2    |   0    |
| Cell 3  |   0    |   0    |   4    |


**Notes:**

How many genes are encoded in human genome and captured by scRNA-seq?  
between 20K and 40k (it depends on the diffintion of gene.protein encoding genes (20K), Non-coding RNA genes (18K).



# **Geneformer: Single-Cell Foundation Model**

Geneformer is a family of single-cell foundation models based on **transformer architecture**, specifically designed to represent **single-cell RNA sequencing (scRNA-seq) data**, where the data is structured as **cell x gene** matrices.


### Geneformer
Geneformer is a foundation transformer model pretrained on Genecorpus-30M, a corpus comprised of ~30 million single cell transcriptomes from a broad range of human tissues. Each single cell’s transcriptome is presented to the model as a rank value encoding where genes are ranked by their expression in that cell normalized by their expression across the entire Genecorpus-30M. Geneformer displays both zero-shot capabilities as well as fine-tuning tasks relevant to chromatin and network dynamics.
- 📄 [Paper](https://www.nature.com/articles/s41586-023-06139-9): The paper that made it into Nature!


### 🧬 About Helical:
Helical provides an open-source framework for and gathers state-of-the-art pre-trained genomics and transciptomics bio foundation models. Still work in progress, but we look forward to interacting with the community to build meaningful things. 
- [Github](https://github.com/helicalAI/helical-package/issues). On our github in the issue section
  
---

### **Pre-training data Overview**

V1 = Genecorpus: 30M cells
V2= Genecorpus-103M, Genecorpus-95M (exclude highly mutational cells), cancer (14M)

### **Model Design Overview**

   - **Pretraining Data**: V1 = Genecorpus: 30M cells
                            V2= Genecorpus-103M, Genecorpus-95M (exclude highly mutational cells), cancer (14M)
   - **Tokenizer Size**: Adjustable based on the model variant.
   - **Input Shape**: The input is represented as `(S, Xi)`, where `S` refers to the number of cells and `Xi` refers to the number of genes (e.g., 2048 or 4096 genes per cell).
   - **Embedding Dimensions** (`d_model`): Typically, `256` embedding dimensions, with a **feedforward network size** of `512`.
   - **Layers**: Geneformer models range from **6 to 20 layers** of transformer blocks.
   - **Encoder/Decoder Architecture**: The model primarily uses an **encoder-only** architecture with multi-head self-attention.
   - **Attention Heads**: Uses **4 attention heads** for multi-head self-attention layers.

---

### **Tokenizer and Data Processing**

Geneformer uses **Ensembl IDs** for tokenization. This enables consistent gene representation and facilitates model interpretability across different gene sets.

1. **Tokenization**:
   - Gene names are mapped into **Ensembl IDs** using the `pyensembl` package.
   - Each normalized gene expression vector is converted to **tokenized rank value encoding**, which is a process of transforming the gene expression data into a numerical format that can be fed into the model.

2. **Genomic Data Processing**:
   - The model processes the genomic data by mapping genes to their corresponding Ensembl IDs. 

3. **Input to Output Transformation**:
   - **Input**: An AnnData object containing the scRNA-seq data is transformed into a Hugging Face **Dataset object**, which is compatible with the model.
   - **Output**: The processed data is tokenized and passed through the model for embedding extraction.

---

###  **Embeddings**:

The model processes input data into an embedding tensor of shape `(S, Xi)`, where:
   - `S` represents the number of cells.
   - `Xi` represents the number of genes per cell (which can be fixed at 2048 or 4096 genes depending on the model).



---

### **Attention Mechanism**

Geneformer uses a **multi-head self-attention mechanism**, which enables the model to focus on different aspects of the input sequence at different positions. The number of attention heads (e.g., **4** heads) allows the model to capture more complex relationships and dependencies between genes across cells.

---

### **Model Architecture**

- **Encoder-Only Architecture**: Geneformer uses an encoder-only architecture, which is ideal for tasks like **embedding generation** and **sequence classification**. This architecture allows the model to learn representations of gene expression sequences effectively.
  
- **No Decoder**: The model does not use a decoder, making it suitable for tasks such as embedding extraction and sequence-based prediction rather than autoregressive generation.

---

### **Applications and Outputs**

- **Embeddings**: The model is designed to compute different embeddings for each chosen `emb_mode`, depending on the user’s task. For instance, users can obtain:

- **Embedding Modes** (`emb_mode`):
   - **`cell`**: Returns **mean embeddings** across all tokens in a cell, providing a holistic view of the cell’s gene expression profile.
   - **`gene`**: Returns embeddings for each gene token, along with corresponding **Ensembl IDs**. This mode helps focus on individual gene-level information.
   - **`cls`**: Returns embeddings of the **CLS token**, which is a special token representing the entire sequence and capturing the global context of the input sequence.

  - Global cell embeddings (via the `cell` mode),
  - Gene-specific embeddings (via the `gene` mode),
  - Sequence-level embeddings (via the `cls` mode).
  
- **Output**: The output of the model is a **tokenized dataset**, which is processed into a **numpy array** containing the embeddings. These embeddings can then be used for downstream tasks such as:
  - **Gene expression analysis**,
  - **Cell-type classification**,
  - **Clustering**, and more.

---

### **End-to-End Pipeline**

1. **Input Data**: An AnnData object containing single-cell RNA-seq data.
You can download annadata objects from https://cellxgene.cziscience.com/datasets
scanpy is a python package for processing annadata object (https://scanpy.readthedocs.io/en/stable/index.html)  
3. **Data Transformation**: Convert the AnnData object into a Hugging Face dataset object.
4. **Embedding Computation**: Use the transformer model to compute the embeddings for the given cells and genes.
5. **Application**: Use the embeddings for tasks like gene classification, clustering, and analysis.

By leveraging the **Geneformer** architecture, users can analyze large-scale single-cell genomic data, uncovering patterns and relationships within the gene expression profiles of individual cells.

---

This model offers a robust framework for extracting meaningful insights from scRNA-seq data, enabling high-throughput, scalable analysis in genomics.

Upload scRNA-seq in annadata format

![AnnData Schema](../images/anndata_schema.svg)



In [ ]:
import anndata as ad


ann_data = ad.read_h5ad("../yolksac_human.h5ad")

print(ann_data)


In [ ]:
print(ann_data.obs.head())

In [ ]:
print(ann_data.var.head())

In [ ]:
print(ann_data.var.index)

print(ann_data.X[:5,:5])

# Helical framework procides thre prepared codes for Geneformer

1- prpeare configuration
2- run the model
3- fine tuning

In [ ]:
from helical.models.geneformer import Geneformer, GeneformerConfig

# CONFIGURATION PARAMETERS

## Parameters
    ----------
###   model_name : Literal["gf-6L-30M-i2048", "gf-12L-30M-i2048", "gf-12L-95M-i4096", "gf-20L-95M-i4096", "gf-12L-95M-i4096-CLcancer"], optional, default="gf-12L-30M-i2048"
        The name of the model.
### batch_size : int, optional, default = 24
        The batch size
###    emb_layer : int, optional, default = -1
        The embedding layer
###    emb_mode : Literal["cls", "cell", "gene"], optional, default="cell"
        The embedding mode to use. "cls" is only available for Geneformer v2 models, returning the embeddings of the cls token.
        For cell level embeddings, a mean across all embeddings excluding the cls token is returned.
        For gene level embeddings, each gene token embedding is returned along with the corresponding ensembl ID.
###    device : Literal["cpu", "cuda"], optional, default="cpu"
        The device to use. Either use "cuda" or "cpu".
###    accelerator : bool, optional, default=False
        The accelerator configuration. By default same device as model.
###    nproc: int, optional, default=1
        Number of processes to use for data processing.
###    custom_attr_name_dict : dict, optional, default=None
        A dictionary that contains the names of the custom attributes to be added to the dataset.
        The keys of the dictionary are the names of the custom attributes, and the values are the names of the columns in adata.obs.
        For example, if you want to add a custom attribute called "cell_type" to the dataset, you would pass custom_attr_name_dict = {"cell_type": "cell_type"}.
        If you do not want to add any custom attributes, you can leave this parameter as None.

In [ ]:
# Example configuration
model_name = "gf-12L-95M-i4096"
batch_size = 20
emb_layer = -1
emb_mode = "gene" # or "cell", or "cls"
device = "cpu"  # or "cuda" for GPU
accelerator = True
nproc = 4

# Example configuration
model_config = GeneformerConfig(    model_name=model_name,
    batch_size=batch_size,
    emb_layer=emb_layer,
    emb_mode=emb_mode,
    device=device,
    accelerator=accelerator,
    nproc=nproc
)



# Geneformer class

1- process_data
2- get_embedding

In [ ]:
geneformer_v2 = Geneformer(model_config)

# data preprocessing

## Convert each cell into words [ 124,22,33,1,555, .......]

1- map gene names to gene ids
2- map gene ids to tokenization dictionary
3- rank genes
4- truncate 
convert annadata object into a tokenized data in hugging face format
## map gene names to ensembl names

## Tokenize:
    Convert normalized gene expression vector to tokenized rank value encoding.

Max input size of model to truncate input to.
            | For the 30M model series, should be 2048. For the 95M model series, should be 4096.
        special_token : bool = True

special_token : bool = True
            | Adds CLS token before and EOS token after rank value encoding.
            | For the 30M model series, should be False. For the 95M model series, should be True.


Normalize expression using gene medians

Rank genes by normalized expression per cell

Convert genes to tokens using Ensembl ID → Token dictionary

Add special tokens (CLS/EOS) if needed for model type

Output: Tokenized dataset ready for transformer-based modeling
#############################################

Ensembl ID	Token
ENSG00000139618	42
ENSG00000227232	97
ENSG00000198786	185



In [ ]:
#?geneformer_v2.process_data

dataset = geneformer_v2.process_data(ann_data[:100])
print(dataset)

In [ ]:
# generate  embeddings = run the model
embeddings = geneformer_v2.get_embeddings(dataset)
#print("Base model embeddings shape:", embeddings.shape)


In [ ]:
print(f"Number of embeddings: {len(embeddings)}")
print(f"Genes per embedding: {len(embeddings[0])}")
print(f"Values per gene embedding: {len(embeddings[0].iloc[0])}")

How many rows are generated in your embeddings?  
How many genes?  
How many embeddings (numbers, d_model) per gene?  

In [ ]:
print(embeddings)
